In [1]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
from symspellpy import SymSpell, Verbosity
from pyvi import ViTokenizer
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import torch


In [21]:
# Khai báo biến

vector_db_path = "vector_db_store/rag_vector_db"

# Tạo vector db từ một đoạn text
def create_vector_db_from_text():
    raw_txt = """ACID URIC - HUYẾT (Tăng)
          *#* Nam: 40 - 70 mg/l
Khi trên: *#* Nữ: 35 - 60 mg/l
          *#* Trẻ: 15 - 40 mg/l
mg x 5,95 = micromol
- TĂNG ACID URIC - HUYẾT THỂ CHẤT (yếu tố di truyền) ở đàn ông:
* Dẫn đến bệnh gút và bệnh sỏi;
- Gia tăng do ăn quá thừa chất, cuộc sống ít vận động, ăn thừa lipid, uống rượu.
- TĂNG ACID URIC - HUYẾT TRONG NHỮNG TRƯỜNG HỢP SUY THẬN, đặc biệt ở bệnh thận nhiễm độc chì và thai nhiễm độc huyết.
- TĂNG ACID URIC - HUYẾT Ở BỆNH MÁU:
* Bệnh Vaquez:
- Bệnh lách to dạng tủy;
- Bệnh bạch cầu dạng tủy, bệnh bạch cầu cấp; và nhất là acid uric - huyết rất tăng, trong bệnh bạch cầu điều trị bằng hóa liệu pháp tiêu tế bào và liệu pháp corticoit (đôi khi vô niệu do urat kết tủa).
- TĂNG ACID URIC - HUYẾT THỰC SỰ DO THUỐC (chích thuốc làm tăng acid uric - huyết):
- Thuốc salicyle với liều nhỏ: Ethambutol; Corticoit...
* Tiêu tế bào, thuộc trị ung thư; thuốc chống đông cumarinic; nhất là thuốc lợi niệu: thiazid (Diưrilix, Esidrex, u.u..); furosemid (Lasilix, Trofurrt), acid etacrynic (Edecrrne).
- TĂNG ACID URIC - HUYẾT GIẢ (giao thoa trong định lượng)
* Acid ascorbic (Vitamin C); Amidopyrin; Alphamethyl - dopa; L.dopa: Phenacetin; Paracetamol; Cafein; Theophyllin;
- Trong bệnh tích glycogen gan, acid urie - huyết rất cao;
- Bệnh não tăng acid uric - huyết, có tính cách gia đình, kèm với chậm tâm thần và tự hủy hoại, do thiếu enzym (hội chứng Lesch - Nyhan)."""

    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500,
        chunk_overlap=50,
        length_function=len
    )

    chunks = text_splitter.split_text(raw_txt)

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )
    # Đưa vào faiss vector db
    vector_db = FAISS.from_texts(chunks, embeddings)
    vector_db.save_local(vector_db_path)
    return vector_db

create_vector_db_from_text()


/mnt/d/HOPT/Agent_build/Telemed_agent/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [23]:
data_path = "/mnt/d/HOPT/Agent_build/Telemed_agent/data/RAG"

def create_db_from_files():
    # Khai báo loader để quét toàn bộ thư mục
    loader = DirectoryLoader(
        data_path,
        glob= "*.pdf",
        loader_cls= PyPDFLoader
    )
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=50
    )
    chunks = text_splitter.split_documents(document)
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )
    db = FAISS.from_documents(chunks, embeddings)
    db.save_local(vector_db_path)
    return db

create_db_from_files()


/mnt/d/HOPT/Agent_build/Telemed_agent/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
